In [1]:
library(tidyverse)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats


In [2]:
data_path <- '../data/'
cohort_df <- read_csv(paste0(data_path, 'cohort.csv'))
action_df <- read_csv(paste0(data_path, 'action_df.csv'))
train_df <- read_csv(paste0(data_path, 'train_data.csv'))
test_df <- read_csv(paste0(data_path, 'test_data.csv'))

Parsed with column specification:
cols(
  subject_id = col_integer(),
  hadm_id = col_integer(),
  icustay_id = col_integer(),
  intime = col_datetime(format = ""),
  outtime = col_datetime(format = ""),
  suspected_infection_time_poe = col_datetime(format = ""),
  window_start = col_datetime(format = ""),
  window_end = col_datetime(format = ""),
  hospital_expire_flag = col_integer()
)
Parsed with column specification:
cols(
  subject_id = col_integer(),
  hadm_id = col_integer(),
  icustay_id = col_integer(),
  interval_start_time = col_datetime(format = ""),
  interval_end_time = col_datetime(format = ""),
  total_IV = col_double(),
  discrete_IV = col_integer(),
  max_VP = col_double(),
  discrete_pressor = col_integer(),
  discrete_action = col_integer()
)
Parsed with column specification:
cols(
  .default = col_double(),
  subject_id = col_integer(),
  hadm_id = col_integer(),
  icustay_id = col_integer(),
  interval_start_time = col_datetime(format = ""),
  interval_end_time = 

In [3]:
outcome_df <- cohort_df %>% group_by(subject_id, 
                                     hadm_id, 
                                     icustay_id)

In [4]:
train_df_action <- train_df %>% inner_join(select(action_df, subject_id, 
                                                   hadm_id, 
                                                   icustay_id, 
                                                   interval_start_time, 
                                                   interval_end_time,
                                                   discrete_action)) %>%
                                inner_join(cohort_df)

test_df_action <- test_df %>% inner_join(select(action_df, subject_id, 
                                                   hadm_id, 
                                                   icustay_id, 
                                                   interval_start_time, 
                                                   interval_end_time,
                                                   discrete_action)) %>%
                                inner_join(cohort_df)

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")
Joining, by = c("subject_id", "hadm_id", "icustay_id")
Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")
Joining, by = c("subject_id", "hadm_id", "icustay_id")


In [5]:
temp_train <- train_df_action %>% group_by(subject_id, 
                             hadm_id, 
                             icustay_id,
                             hospital_expire_flag) %>%
                    summarise(interval_start_time = max(interval_start_time)) %>%
                    mutate(reward = ifelse(hospital_expire_flag, -15, 15))

train_df_action_reward <- train_df_action %>% 
                                left_join(temp_train) %>%
                                replace_na(list(reward = 0))

temp_test <- test_df_action %>% group_by(subject_id, 
                             hadm_id, 
                             icustay_id,
                             hospital_expire_flag) %>%
                    summarise(interval_start_time = max(interval_start_time)) %>%
                    mutate(reward = ifelse(hospital_expire_flag, -15, 15))

test_df_action_reward <- test_df_action %>% 
                                left_join(temp_test) %>%
                                replace_na(list(reward = 0))

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "hospital_expire_flag")
Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "hospital_expire_flag")


In [6]:
temp2_train <- train_df_action_reward %>% 
                                mutate(row_id = 1:nrow(train_df_action_reward) - 1) %>% 
                                arrange(subject_id, hadm_id, icustay_id, interval_start_time, row_id) %>%
                                group_by(subject_id, hadm_id, icustay_id) %>%
                                mutate(row_id_next = ifelse(row_id == max(row_id), row_id, row_id + 1))

temp2_test <- train_df_action_reward %>% 
                                mutate(row_id = 1:nrow(train_df_action_reward) - 1) %>% 
                                arrange(subject_id, hadm_id, icustay_id, interval_start_time, row_id) %>%
                                group_by(subject_id, hadm_id, icustay_id) %>%
                                mutate(row_id_next = ifelse(row_id == max(row_id), row_id, row_id + 1))


In [7]:
final_df_train <- temp2_train %>% select(
                             -exclude, 
                             -intime, 
                             -outtime, 
                             -window_start,
                             -window_end,
                             -suspected_infection_time_poe
                             )

final_df_test <- temp2_test %>% select(
                             -exclude, 
                             -intime, 
                             -outtime, 
                             -window_start,
                             -window_end,
                             -suspected_infection_time_poe
                             )

In [10]:
head(final_df_train)

subject_id,hadm_id,icustay_id,interval_start_time,interval_end_time,ALBUMIN,ANION GAP,BANDS,BICARBONATE,BILIRUBIN,⋯,qsofa_sysbp_score,qsofa_gcs_score,qsofa_resprate_score,elixhauser_hospital,blood_culture_positive,discrete_action,hospital_expire_flag,reward,row_id,row_id_next
4,185777,294638,2191-03-15 12:00:00,2191-03-15 16:00:00,-0.05746277,0.7674253,-0.2986458,0.04208446,0.04837396,⋯,0.5402321,-0.5779918,0.3918894,1.006947,1.458774,0,0,0,0,1
4,185777,294638,2191-03-15 20:00:00,2191-03-16 00:00:00,-0.05746277,0.7674253,-0.2986458,0.04208446,0.04837396,⋯,0.5402321,-0.5779918,0.3918894,1.006947,1.458774,0,0,0,1,2
4,185777,294638,2191-03-16 00:00:00,2191-03-16 04:00:00,-0.05746277,0.7674253,-0.2986458,0.04208446,0.04837396,⋯,0.5402321,-0.5779918,0.3918894,1.006947,1.458774,2,0,0,2,3
4,185777,294638,2191-03-16 04:00:00,2191-03-16 08:00:00,-0.48527552,0.2709276,-0.2986458,-0.59964321,-0.02016620,⋯,0.5402321,-0.5779918,0.3918894,1.006947,1.458774,4,0,0,3,4
4,185777,294638,2191-03-16 08:00:00,2191-03-16 12:00:00,-0.48527552,0.2709276,-0.2986458,-0.59964321,-0.02016620,⋯,0.5402321,-0.5779918,0.3918894,1.006947,1.458774,1,0,0,4,5
4,185777,294638,2191-03-16 12:00:00,2191-03-16 16:00:00,-0.48527552,0.2709276,-0.2986458,-0.59964321,-0.02016620,⋯,0.5402321,-0.5779918,0.3918894,1.006947,1.458774,3,0,0,5,6


In [8]:
names(final_df_train)
names(final_df_test)

[1] "subject_id"             "hadm_id"                "icustay_id"            
 [4] "interval_start_time"    "interval_end_time"      "ALBUMIN"               
 [7] "ANION GAP"              "BANDS"                  "BICARBONATE"           
[10] "BILIRUBIN"              "BUN"                    "CHLORIDE"              
[13] "CREATININE"             "DiasBP"                 "Glucose"               
[16] "GLUCOSE"                "HeartRate"              "HEMATOCRIT"            
[19] "HEMOGLOBIN"             "INR"                    "LACTATE"               
[22] "MeanBP"                 "PaCO2"                  "PLATELET"              
[25] "POTASSIUM"              "PT"                     "PTT"                   
[28] "RespRate"               "SODIUM"                 "SpO2"                  
[31] "SysBP"                  "TempC"                  "WBC"                   
[34] "age"                    "is_male"                "race_white"            
[37] "race_black"             "race_hispanic"          "race_other"            
[40] "height"                 "weight"                 "vent"                  
[43] "sofa"                   "lods"                   "sirs"                  
[46] "qsofa"                  "qsofa_sysbp_score"      "qsofa_gcs_score"       
[49] "qsofa_resprate_score"   "elixhauser_hospital"    "blood_culture_positive"
[52] "discrete_action"        "hospital_expire_flag"   "reward"                
[55] "row_id"                 "row_id_next"

[1] "subject_id"             "hadm_id"                "icustay_id"            
 [4] "interval_start_time"    "interval_end_time"      "ALBUMIN"               
 [7] "ANION GAP"              "BANDS"                  "BICARBONATE"           
[10] "BILIRUBIN"              "BUN"                    "CHLORIDE"              
[13] "CREATININE"             "DiasBP"                 "Glucose"               
[16] "GLUCOSE"                "HeartRate"              "HEMATOCRIT"            
[19] "HEMOGLOBIN"             "INR"                    "LACTATE"               
[22] "MeanBP"                 "PaCO2"                  "PLATELET"              
[25] "POTASSIUM"              "PT"                     "PTT"                   
[28] "RespRate"               "SODIUM"                 "SpO2"                  
[31] "SysBP"                  "TempC"                  "WBC"                   
[34] "age"                    "is_male"                "race_white"            
[37] "race_black"             "race_hispanic"          "race_other"            
[40] "height"                 "weight"                 "vent"                  
[43] "sofa"                   "lods"                   "sirs"                  
[46] "qsofa"                  "qsofa_sysbp_score"      "qsofa_gcs_score"       
[49] "qsofa_resprate_score"   "elixhauser_hospital"    "blood_culture_positive"
[52] "discrete_action"        "hospital_expire_flag"   "reward"                
[55] "row_id"                 "row_id_next"

In [9]:
write_csv(final_df_train, paste0(data_path, 'train_state_action_reward_df.csv'))
write_csv(final_df_test, paste0(data_path, 'test_state_action_reward_df.csv'))